# RAG
在这篇notebook中，我们将实现一个标准的RAG流程，并且评估标准RAG算法的性能。
标准RAG一共分为三个流程，检索（Retrieval, R）、增强（Augmented, A）、生成（Generation, G）。
- **检索**：使用嵌入模型，根据问题检索文档中的多个相关文本块。
- **增强**：拼接上下文信息，增强信息的表达。
- **生成**：根据上下文信息生成回复。

RAG是由 **Facebook AI Research（现 Meta AI）** 的一个团队在一篇名为 《**Retrieval-Augmented Generation for Knowledge-Intensive NLP Tasks**》的论文中正式提出的。
> https://arxiv.org/abs/2005.11401

其核心特点是 端到端的可微调。这意味着检索器（Retriever）和生成器（Generator）被视为一个统一的、可以共同训练的系统。
如今我们谈论的RAG，是一种更加模块化和实用化的架构。
- 检索器: 通常是一个独立的向量数据库和嵌入模型，它的任务只有一个：根据查询返回最相关的文本块。
- 生成器: 通常是一个强大的、通用的、预训练好的LLM。

![](figures/01%20RAG/流程图.png)

接下来，我们开始对标准的RAG进行评估。

首先需要导入相关的代码库。

In [1]:
from rag_evaluate import Embedder, LLMJudge, load_data, ReplyModel
import warnings
warnings.filterwarnings('ignore')  # 忽略所有的警告

加载评估的相关数据。

In [2]:
documents, QA = load_data(data_dir='data')

在进行RAG前，首先要将文档进行分块。
原因如下：
1. 如果文档太长，嵌入模型难以处理。
2. 一个完整的文档中，可能只有部分语句是与问题相关的，因此需要对文档进行切块。
3. LLM尽量避免输入太长的token，输入token太长容易造成无法抓住重点。

我们使用最简单的分块策略，即按照自然段进行划分。
在划分时，需要指定一个 `chunk_size`，用于指定每一个文本模块的大小。
文本块太短容易造成信息不充分。
文本块太长容易造成信息的冗余。
在下面的代码中，有些自然段的长度可能超过chunk_size，这些内容暂时不做处理。我们只对长度较短的自然段进行拼接。

In [3]:
def chunk_documents(documents, chunk_size):
    chunks = []
    for document in documents:
        # 按段落分割，并过滤掉空段落
        paragraphs = [p.strip() for p in document.split("\n") if p.strip()]

        current_chunk = ""
        for i, paragraph in enumerate(paragraphs):
            current_chunk += paragraph
            if len(current_chunk) >= chunk_size:
                chunks.append(current_chunk)
                current_chunk = ""
    
    return chunks

指定文本块的最小长度为512，并提取所有的问题与正确答案。

In [4]:
chunks = chunk_documents(documents, 512)
queries = [qa["question"] for qa in QA['data']]
answers_gt = [qa["answer"] for qa in QA['data']]

加载向量化工具，并对所有的**文本块**和**问题**进行向量化。

In [5]:
embedder = Embedder()
chunk_embeddings = embedder.embed(chunks)
query_embeddings = embedder.embed(queries)

接下来，计算所有问题与所有文本块之间的相似度矩阵。

通过相似度矩阵，我们可以找到每个**问题**与每个**文本块**之间的**相似度**，从而找到某一个问题与哪些文本块存在着关联。

在这里，我们需要指定一个`k值`，通过k值确定需要的文本块数量。

若k值太大，会造成文本模块过多，引入无关文本块，同时会造成输入token数量的爆炸💥。

若K值太小，则会导致信息量的缺失，可能无法找到相关的文本块。

在代码中，我们将得到每个问题相关文本块的索引（`top_k_indices`），后续可以通过索引找到相关的文本块。

In [6]:
similarity = embedder.similarity(query_embeddings, chunk_embeddings)
k = 3
top_k_indices = embedder.get_top_k(similarity, k)

在检索到相关的文本块后，需要使用LLM进行回复。

相关的提示词模板为：
系统提示词为：
```
你是一个问答机器人。请严格根据下面提供的“参考文档”来回答问题。
如果文档中的信息不足以回答，直接回复“根据提供的文档，我无法回答该问题。”
```

用户提示词为：
```
参考文档:
---
{context_str}
---

问题: {query}
```

In [7]:
llm = ReplyModel()
answers = []
for i, query in enumerate(queries):
    retrieved_chunks = []
    for doc_idx in top_k_indices[i]:
        retrieved_chunks.append(chunks[doc_idx])
        
    answer = llm.answer(query, retrieved_chunks)
    answers.append(answer)
    print(query)
    print(answer)
    print("=" * 30)

在后训练过程中，为什么Qwen3模型在经过“思考模式融合”和“通用RL”阶段后，在AIME’24和LiveCodeBench等具有挑战性的任务上性能反而有所下降？
根据提供的文档，Qwen3模型在经过“思考模式融合”和“通用RL”阶段后，在AIME’24和LiveCodeBench等具有挑战性的任务上性能下降的原因是：模型在更广泛的通用任务上训练，这可能损害了其处理复杂问题的专业能力。文档中提到，这种退化是由于模型在更广泛的通用任务上训练，可能影响了其处理复杂问题的专业能力，因此选择接受这种性能权衡以增强模型的整体多功能性。
在针对轻量级模型的“强到弱蒸馏”管道中，在线蒸馏（Online Distillation）阶段的具体实现方式是什么？
根据提供的文档，"强到弱蒸馏"管道分为两个主要阶段：（1）离线蒸馏和（2）在线训练。然而，文档中并未详细说明在线蒸馏（Online Distillation）阶段的具体实现方式。因此，无法提供在线蒸馏阶段的具体实现方式。
Qwen3的开发团队在预训练数据筛选和模型后训练阶段，具体实施了哪些伦理审查（Ethical Review）流程来确保模型的安全性并减少偏见？
根据提供的文档，我无法回答该问题。文档中未提及Qwen3的开发团队在预训练数据筛选和模型后训练阶段具体实施了哪些伦理审查（Ethical Review）流程来确保模型的安全性并减少偏见。
与Qwen2.5-MoE相比，Qwen3的MoE模型在架构设计上引入了哪些关键的改变？
与Qwen2.5-MoE相比，Qwen3的MoE模型在架构设计上引入了以下关键的改变：

1. **移除了共享专家**：Qwen3-MoE设计中不包含共享专家，而Qwen2.5-MoE则包含共享专家。

2. **细粒度专家分割**：Qwen3-MoE实现了细粒度专家分割，这是基于Dai等人（2024）的工作。

3. **全局批处理负载平衡损失**：Qwen3-MoE采用了全局批处理负载平衡损失（Qiu等人，2025），以促进专家专业化。

4. **激活参数的优化**：实验结果表明，Qwen3 MoE基础模型仅使用1/5的激活参数即可实现与Qwen3密集型基础模型相似的性能，并且可以使用更少的激活参数和总参数超越Qwen2.5 MoE基础模型。这表明Qwen3 MoE在训练和推理成本方面具有显著

最后一步，使用另一个性能较强的模型，对答案进行评估。

In [8]:
judge = LLMJudge()
judge.evaluate(queries, answers_gt, answers)

1 正在评估问题：在后训练过程中，为什么Qwen3模型在经过“思考模式融合”和“通用RL”阶段后，在AIME’24和LiveCodeBench等具有挑战性的任务上性能反而有所下降？
--------------------
{'scores': {'Correctness': 5, 'Completeness': 4, 'Clarity & Conciseness': 5}, 'reasoning': "生成答案准确捕捉了正确答案的核心因果关系（通用任务训练损害专业能力），且完整复现了'多功能性权衡'这一关键结论。但在完整性上扣1分，因为未明确提及原文档中'退化'（degradation）这一专业表述，且未具体指明AIME’24和LiveCodeBench这两个任务场景。语言表达简洁流畅，逻辑结构清晰，无冗余信息。", 'final_score': 4.666666666666667}
2 正在评估问题：在针对轻量级模型的“强到弱蒸馏”管道中，在线蒸馏（Online Distillation）阶段的具体实现方式是什么？
--------------------
{'scores': {'Correctness': 1, 'Completeness': 1, 'Clarity & Conciseness': 4}, 'reasoning': '生成答案声称在线蒸馏阶段涉及学生模型生成在线序列并与教师模型logits对齐，但根据正确答案，文档并未提供在线蒸馏的具体实现方式。因此，生成答案的内容与正确答案相悖，存在严重错误，导致正确性和完整性得分极低。表达上较为清晰，但内容完全偏离事实，仅在语言层面保持了一定质量。', 'final_score': 2.0}
3 正在评估问题：Qwen3的开发团队在预训练数据筛选和模型后训练阶段，具体实施了哪些伦理审查（Ethical Review）流程来确保模型的安全性并减少偏见？
--------------------
{'scores': {'Correctness': 5, 'Completeness': 1, 'Clarity & Conciseness': 5}, 'reasoning': '生成答案正确复现了【正确答案】的核心结论（无法回答），因此在正确性上得5分。然而，生成答案未包含【正确答案】中明确指出的‘文档中未提及

从结果可以看出，RAG最终的评分为：3.50分（满分5分）